In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.utils import image_dataset_from_directory, to_categorical
import os, glob
from tifffile import imread, imwrite
import numpy as np
import matplotlib.pyplot as plt


2022-08-16 21:16:17.443716: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-16 21:16:17.443793: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


import pandas as pd 
labels = pd.read_pickle(f'{os.environ["HOME"]}/Dokumente/AML Projekt/labels_pkl/labels.pkl')
for i in range(len(labels)): ##über len(labels)
    im_old = imread(f'{os.environ["HOME"]}/Dokumente/AML Projekt/cropped_labels/{labels["img_folder"][i]}/{os.path.splitext(labels["img_name"][i])[0]}.tif')
    im_new = to_categorical(im_old)
    imwrite(f'{os.environ["HOME"]}/Dokumente/AML Projekt/categorical_labels/{labels["img_folder"][i]}/{os.path.splitext(labels["img_name"][i])[0]}.tif',im_new)

In [2]:
epochs = 100
num_batches = 250
batch_size = 8

patch_size = 256

num_classes = 44
classes = np.arange(44)
scores = [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 24., 25., 26.,
       27., 28., 30., 32., 33., 34., 36., 38., 39., 40., 42., 45., 48.,
       50., 51., 54., 57., 60.]

class MyLRSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, initial_learning_rate):
    self.initial_learning_rate = initial_learning_rate

  def __call__(self, epoch):
     return (self.initial_learning_rate - epoch/epochs)**0.9

def DiceLoss(targets, inputs, smooth=1e-6):
    
    #flatten label and prediction tensors
    inputs = keras.backend.flatten(inputs)
    targets = keras.backend.flatten(targets)
    
    intersection = keras.backend.sum(keras.backend.dot(targets, inputs))
    dice = (2*intersection + smooth) / (keras.backend.sum(targets) + keras.backend.sum(inputs) + smooth)
    return 1 - dice + keras.losses.CategoricalCrossentropy()(targets,inputs)

loss = DiceLoss

optimizer = keras.optimizers.SGD(MyLRSchedule(1.),momentum=0.99,nesterov=True, name='SGD')



In [3]:
def load_label(path_to_label):

    label = imread(path_to_label)
    
    for i, score in enumerate(scores):
        label[label == score] = i

    label = to_categorical(label,num_classes)

    return label

In [4]:
def load_training_data(img_dir, label_dir):
    
    # get filenames
    img_names = sorted(glob.glob(img_dir+'/**/*'+'.JPG', recursive=True))
    label_names = sorted(glob.glob(label_dir+'/**/*'+'.tif', recursive=True))


    # load first label
    labels = np.zeros((batch_size,patch_size,patch_size,num_classes))

    # Initalise mini batch
    images = np.zeros((batch_size,patch_size,patch_size,3))

    for i, (img_name, label_name) in enumerate(zip(img_names[0:batch_size], label_names[0:batch_size])):

        # append label
        labels[i] = load_label(label_name)[400-128:400+128,400-128:400+128]

        # append image
        new_img = plt.imread(img_name)[400-128:400+128,400-128:400+128]
        new_img = new_img.astype(np.float32)
        new_img -= np.amin(new_img)
        new_img /= np.amax(new_img)
        images[i] = new_img
        

    return images, labels

In [5]:
#load the data to train and validate

train_img_path = '/home/jacob-relle/Dokumente/AML Projekt/cropped_images/800/d1_02_04_2020'
train_label_path = '/home/jacob-relle/Dokumente/AML Projekt/cropped_labels/d1_02_04_2020'

val_img_path = '/home/jacob-relle/Dokumente/AML Projekt/cropped_images/800/d1_02_06_2020'
val_label_path = '/home/jacob-relle/Dokumente/AML Projekt/cropped_labels/d1_02_06_2020'

x_train, y_train = load_training_data(train_img_path,train_label_path)
x_val, y_val = load_training_data(val_img_path,val_label_path)

In [6]:
y_train.shape

(8, 256, 256, 44)

In [7]:
from keras.layers import MaxPooling2D, Conv2D, Conv2DTranspose, concatenate ,BatchNormalization, LeakyReLU

Norm = BatchNormalization
LReLU = LeakyReLU
Pool = MaxPooling2D

def NN_UNet():
    inputs = keras.Input(shape=(patch_size,patch_size,3))

    #Downsampeling
    x = Conv2D(32,3,padding='same')(inputs)
    x = Norm()(x)
    x = LReLU(alpha= 0.01)(x)

    x = Conv2D(32,3,padding='same')(x)
    x = Norm()(x)
    l1 = LReLU(alpha= 0.01)(x)

    x = Pool(pool_size= 2, strides= 2, padding='valid')(l1)

    x = Conv2D(64,3,padding='same')(x)
    x = Norm()(x)
    x = LReLU(alpha= 0.01)(x)

    x = Conv2D(64,3,padding='same')(x)
    x = Norm()(x)
    l2 = LReLU(alpha= 0.01)(x)

    x = Pool(pool_size= 2, strides= 2, padding='valid')(l2)

    x = Conv2D(128,3,padding='same')(x)
    x = Norm()(x)
    x = LReLU(alpha= 0.01)(x)

    x = Conv2D(128,3,padding='same')(x)
    x = Norm()(x)
    l3 = LReLU(alpha= 0.01)(x)
    
    x = Pool(pool_size= 2, strides= 2, padding='valid')(l3)

    x = Conv2D(256,3,padding='same')(x)
    x = Norm()(x)
    x = LReLU(alpha= 0.01)(x)

    x = Conv2D(256,3,padding='same')(x)
    x = Norm()(x)
    l4 = LReLU(alpha= 0.01)(x)
    '''
    x = Pool(pool_size= 2, strides= 2, padding='valid')(l4)
    
    #Bottleneck
    x = Conv2D(512,3,padding='same')(x)
    x = Norm()(x)
    x = LReLU(alpha= 0.01)(x)

    x = Conv2D(512,3,padding='same')(x)
    x = Norm()(x)
    l5 = LReLU(alpha= 0.01)(x)
    
    #Upsampleing
    x = Conv2DTranspose(256,3,2,padding='same')(l5)
    x = concatenate([x,l4])

    x = Conv2D(256,3,padding='same')(x)
    x = Norm()(x)
    x = LReLU(alpha= 0.01)(x)

    x = Conv2D(256,3,padding='same')(x)
    x = Norm()(x)
    x = LReLU(alpha= 0.01)(x)
    '''

    x = Conv2DTranspose(128,3,2,padding='same')(l4)
    x = concatenate([x,l3])
    
    x = Conv2D(128,3,padding='same')(x)
    x = Norm()(x)
    x = LReLU(alpha= 0.01)(x)

    x = Conv2D(128,3,padding='same')(x)
    x = Norm()(x)
    x = LReLU(alpha= 0.01)(x)
    
    x = Conv2DTranspose(64,3,2,padding='same')(x)
    x = concatenate([x,l2])

    x = Conv2D(64,3,padding='same')(x)
    x = Norm()(x)
    x = LReLU(alpha= 0.01)(x)

    x = Conv2D(64,3,padding='same')(x)
    x = Norm()(x)
    x = LReLU(alpha= 0.01)(x)

    x = Conv2DTranspose(32,3,2,padding='same')(x)
    x = concatenate([x,l1])

    x = Conv2D(32,3,padding='same')(x)
    x = Norm()(x)
    x = LReLU(alpha= 0.01)(x)

    x = Conv2D(32,3,padding='same')(x)
    x = Norm()(x)
    x = LReLU(alpha= 0.01)(x)

    x = Conv2D(44,1)(x)
    outputs = keras.activations.softmax(x)

    model = keras.Model(inputs,outputs,name='U-Net')
    return model

UNet = NN_UNet()
UNet.summary()


2022-08-16 21:16:24.813978: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-08-16 21:16:24.814010: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Murasame): /proc/driver/nvidia/version does not exist
2022-08-16 21:16:24.815174: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [8]:
UNet.compile(
    loss = keras.losses.CategoricalCrossentropy(),
    optimizer = optimizer,
    metrics=["accuracy"]    
)

history = UNet.fit(x=x_train,y=y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_val,y_val))
#test_scores = model.evaluate(x_test,y_test,verbose=2) 

Epoch 1/100
1/1 [==============================] - 12s 12s/step - loss: 4.0632 - accuracy: 0.0272 - val_loss: 7.0680 - val_accuracy: 0.0340
Epoch 2/100
1/1 [==============================] - 8s 8s/step - loss: 3.6788 - accuracy: 0.0576 - val_loss: 10.7765 - val_accuracy: 0.0592
Epoch 3/100
1/1 [==============================] - 9s 9s/step - loss: 3.0806 - accuracy: 0.1296 - val_loss: 26.0938 - val_accuracy: 0.0598
Epoch 4/100
1/1 [==============================] - 10s 10s/step - loss: 2.6473 - accuracy: 0.1418 - val_loss: 51.1291 - val_accuracy: 0.0573
Epoch 5/100
1/1 [==============================] - 9s 9s/step - loss: 2.4483 - accuracy: 0.1497 - val_loss: 63.5048 - val_accuracy: 0.0587
Epoch 6/100
1/1 [==============================] - 9s 9s/step - loss: 2.3386 - accuracy: 0.1477 - val_loss: 121.9817 - val_accuracy: 0.0449
Epoch 7/100
1/1 [==============================] - 10s 10s/step - loss: 2.3139 - accuracy: 0.2228 - val_loss: 274.8125 - val_accuracy: 0.0528
Epoch 8/100
1/1 [===

KeyboardInterrupt: 

In [16]:
test = UNet.predict(x_train)
test[0]

1/1 [==============================] - 1s 1s/step


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [ ]:
import pandas as pd
import os 

pd.read_pickle(f'{os.environ["HOME"]}/Dokumente/deepdart_aml/labelswithscore.pkl')

,img_folder,img_name,bbox,xy,score
0,d1_02_04_2020,IMG_1081.JPG,"[2297, 3949, 676, 2328]","[[0.4350282485875706, 0.1285310734463277], [0....",0.0
1,d1_02_04_2020,IMG_1082.JPG,"[2267, 3803, 759, 2295]","[[0.44162326388888884, 0.12738715277777776], [...",20.0
2,d1_02_04_2020,IMG_1083.JPG,"[2267, 3803, 759, 2295]","[[0.44162326388888884, 0.12738715277777776], [...",29.0
3,d1_02_04_2020,IMG_1084.JPG,"[2267, 3803, 759, 2295]","[[0.44162326388888884, 0.12738715277777776], [...",69.0
4,d1_02_04_2020,IMG_1085.JPG,"[2267, 3803, 759, 2295]","[[0.44162326388888884, 0.12738715277777776], [...",7.0
...,...,...,...,...,...
16045,d2_04_05_2020,DSC_0507.JPG,"[200, 2849, 969, 3618]","[[0.4369087750204756, 0.1419047415543067], [0....",27.0
16046,d2_04_05_2020,DSC_0508.JPG,"[200, 2849, 969, 3618]","[[0.4369087750204756, 0.1419047415543067], [0....",34.0
16047,d2_04_05_2020,DSC_0509.JPG,"[200, 2849, 969, 3618]","[[0.4369087750204756, 0.1419047415543067], [0....",1.0
16048,d2_04_05_2020,DSC_0510.JPG,"[200, 2849, 969, 3618]","[[0.4369087750204756, 0.1419047415543067], [0....",21.0
